## 1. Import Necessary Libraries
First, let's import the libraries we'll need for this task.

In [1]:
import pandas as pd
import requests
import gzip
import shutil
import os

## 2. Define File Paths and URL
Next, we'll define the URL for the data and the names of the files we'll be working with.

In [2]:
# URL for the Redfin data
url = "https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/zip_code_market_tracker.tsv000.gz"

# File names for the downloaded and processed data
gzipped_file = 'data/raw/zip_code_market_tracker.tsv000.gz'
unzipped_file = 'data/raw/zip_code_market_tracker.tsv'
# CHANGED: The final raw file is now saved in the data/raw/ directory
final_csv_file = 'data/raw/redfin_raw_data.csv'

## 3. Download the Data
This function will download the file from the URL. It includes a basic check to see if the download was successful.

In [3]:
def download_file(url, filename):
    """Downloads a file from a given URL."""
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Successfully downloaded {filename}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        return False

def decompress_gzip(gzipped_file, unzipped_file):
    """Decompresses a gzipped file."""
    try:
        with gzip.open(gzipped_file, 'rb') as f_in:
            with open(unzipped_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Successfully decompressed {gzipped_file} to {unzipped_file}")
        return True
    except Exception as e:
        print(f"Error decompressing file: {e}")
        return False

## 4. Load and Process the Data
This is the core of the script. It orchestrates the downloading, loading, and merging of the data.

In [4]:
def update_dataset(url, gzipped_file, unzipped_file, final_csv_file):
    """
    Downloads, processes, and updates the Redfin dataset.
    """
    # ADDED: Create the data/raw directory if it doesn't exist
    os.makedirs(os.path.dirname(gzipped_file), exist_ok=True)
    
    if not download_file(url, gzipped_file):
        return

    if not decompress_gzip(gzipped_file, unzipped_file):
        return

    try:
        new_data = pd.read_csv(unzipped_file, sep='\t')
        print("Successfully loaded new data.")
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    if os.path.exists(final_csv_file):
        print("Existing dataset found. Merging with new data.")
        old_data = pd.read_csv(final_csv_file)
        combined_data = pd.concat([old_data, new_data], ignore_index=True)
        # CHANGED: Column names are lowercase in the raw file
        combined_data.drop_duplicates(subset=['REGION', 'PERIOD_BEGIN', 'PERIOD_END'], keep='last', inplace=True)
        print("Successfully merged and removed duplicates.")
    else:
        print("No existing dataset found. Creating a new one.")
        combined_data = new_data

    combined_data.to_csv(final_csv_file, index=False)
    print(f"Raw dataset saved to {final_csv_file}")

    # Clean up temporary downloaded files
    os.remove(gzipped_file)
    os.remove(unzipped_file)
    print("Cleaned up temporary files.")

# --- Step 4: Run the Process ---
update_dataset(url, gzipped_file, unzipped_file, final_csv_file)

Successfully downloaded data/raw/zip_code_market_tracker.tsv000.gz
Successfully decompressed data/raw/zip_code_market_tracker.tsv000.gz to data/raw/zip_code_market_tracker.tsv
Successfully loaded new data.
Existing dataset found. Merging with new data.
Successfully merged and removed duplicates.
Raw dataset saved to data/raw/redfin_raw_data.csv
Cleaned up temporary files.


## 6. Load the Dataset
Inspect the dataset it's shape, types,typos, etc..

In [5]:
# Load the dataset
try:
    df = pd.read_csv('data/raw/redfin_raw_data.csv')
    print("Dataset loaded successfully!")
    print(f"The dataset contains {df.shape[0]:,} rows and {df.shape[1]} columns.")
except FileNotFoundError:
    print("Error: 'redfin_zipcode_data.csv' not found. Please run the data download script first.")

# --- Data Preparation ---

# Convert date columns to datetime objects
df['PERIOD_BEGIN'] = pd.to_datetime(df['PERIOD_BEGIN'])
df['PERIOD_END'] = pd.to_datetime(df['PERIOD_END'])

# For this analysis, we want the most recent data for each zip code.
# Let's find the latest period available in the dataset.
latest_date = df['PERIOD_BEGIN'].max()
print(f"\nMost recent data is from: {latest_date.strftime('%Y-%m-%d')}")

# Create a dataframe with only the most recent data for each zip code
latest_df = df[df['PERIOD_BEGIN'] == latest_date].copy()

Dataset loaded successfully!
The dataset contains 3,097,102 rows and 58 columns.

Most recent data is from: 2025-05-01


In [6]:
# Let's Check the first rows
df.head()

,PERIOD_BEGIN,PERIOD_END,PERIOD_DURATION,REGION_TYPE,REGION_TYPE_ID,TABLE_ID,IS_SEASONALLY_ADJUSTED,REGION,CITY,STATE,...,SOLD_ABOVE_LIST_YOY,PRICE_DROPS,PRICE_DROPS_MOM,PRICE_DROPS_YOY,OFF_MARKET_IN_TWO_WEEKS,OFF_MARKET_IN_TWO_WEEKS_MOM,OFF_MARKET_IN_TWO_WEEKS_YOY,PARENT_METRO_REGION,PARENT_METRO_REGION_METRO_CODE,LAST_UPDATED
0,2014-07-01,2014-09-30,90,zip code,2,15721,False,Zip Code: 37145,NaN,Tennessee,...,0.000000,NaN,NaN,NaN,0.0,0.000000,0.000000,"Nashville, TN",34980,2025-08-01 14:37:32.798 Z
1,2021-10-01,2021-12-31,90,zip code,2,17323,False,Zip Code: 40903,NaN,Kentucky,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"London, KY",30940,2025-08-01 14:37:32.798 Z
2,2021-08-01,2021-10-31,90,zip code,2,30991,False,Zip Code: 72386,NaN,Arkansas,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,"Jonesboro, AR",27860,2025-08-01 14:37:32.798 Z
3,2017-06-01,2017-08-31,90,zip code,2,19279,False,Zip Code: 45764,NaN,Ohio,...,0.142857,NaN,NaN,NaN,0.2,0.088889,0.200000,"Athens, OH",11900,2025-08-01 14:37:32.798 Z
4,2020-09-01,2020-11-30,90,zip code,2,33520,False,Zip Code: 77577,NaN,Texas,...,-0.333333,NaN,NaN,NaN,0.4,0.000000,0.066667,"Houston, TX",26420,2025-08-01 14:37:32.798 Z


In [7]:
# Let's Check The columns
df.columns

Index(['PERIOD_BEGIN', 'PERIOD_END', 'PERIOD_DURATION', 'REGION_TYPE',
       'REGION_TYPE_ID', 'TABLE_ID', 'IS_SEASONALLY_ADJUSTED', 'REGION',
       'CITY', 'STATE', 'STATE_CODE', 'PROPERTY_TYPE', 'PROPERTY_TYPE_ID',
       'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY',
       'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_MOM', 'MEDIAN_LIST_PRICE_YOY',
       'MEDIAN_PPSF', 'MEDIAN_PPSF_MOM', 'MEDIAN_PPSF_YOY', 'MEDIAN_LIST_PPSF',
       'MEDIAN_LIST_PPSF_MOM', 'MEDIAN_LIST_PPSF_YOY', 'HOMES_SOLD',
       'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY', 'PENDING_SALES',
       'PENDING_SALES_MOM', 'PENDING_SALES_YOY', 'NEW_LISTINGS',
       'NEW_LISTINGS_MOM', 'NEW_LISTINGS_YOY', 'INVENTORY', 'INVENTORY_MOM',
       'INVENTORY_YOY', 'MONTHS_OF_SUPPLY', 'MONTHS_OF_SUPPLY_MOM',
       'MONTHS_OF_SUPPLY_YOY', 'MEDIAN_DOM', 'MEDIAN_DOM_MOM',
       'MEDIAN_DOM_YOY', 'AVG_SALE_TO_LIST', 'AVG_SALE_TO_LIST_MOM',
       'AVG_SALE_TO_LIST_YOY', 'SOLD_ABOVE_LIST', 'SOLD_ABOVE_LIST_MOM',
 

In [8]:
# Replace REGION For Zip Code because Region has strings before the actual zipcode number

# Check if 'region' column exists
if 'REGION' in df.columns:
    # Extract the zip code number by splitting the string
    df['ZIP_CODE'] = df['REGION'].str.split(': ').str[1]

    # Convert the new column to a numeric type, forcing errors to become NaN (Not a Number)
    df['ZIP_CODE'] = pd.to_numeric(df['ZIP_CODE'], errors='coerce')

    # Drop the old 'region' column
    df.drop(columns=['REGION'], inplace=True)

    print("Successfully cleaned and renamed the 'region' column to 'ZIP_CODE'.")
    print(df[['ZIP_CODE']].head())
else:
    print("'REGION' column not found. It may have been cleaned already.")

Successfully cleaned and renamed the 'region' column to 'ZIP_CODE'.
   ZIP_CODE
0     37145
1     40903
2     72386
3     45764
4     77577


In [9]:
# Check column types
df.dtypes

PERIOD_BEGIN                      datetime64[ns]
PERIOD_END                        datetime64[ns]
PERIOD_DURATION                            int64
REGION_TYPE                               object
REGION_TYPE_ID                             int64
TABLE_ID                                   int64
IS_SEASONALLY_ADJUSTED                      bool
CITY                                     float64
STATE                                     object
STATE_CODE                                object
PROPERTY_TYPE                             object
PROPERTY_TYPE_ID                           int64
MEDIAN_SALE_PRICE                        float64
MEDIAN_SALE_PRICE_MOM                    float64
MEDIAN_SALE_PRICE_YOY                    float64
MEDIAN_LIST_PRICE                        float64
MEDIAN_LIST_PRICE_MOM                    float64
MEDIAN_LIST_PRICE_YOY                    float64
MEDIAN_PPSF                              float64
MEDIAN_PPSF_MOM                          float64
MEDIAN_PPSF_YOY     

## 7. Check uniqueness
Now, let's ensure that every row in our dataset represents a unique combination of a zip code and a time period.

In [10]:
# Define the key columns that should make a row unique
key_columns = ['ZIP_CODE', 'PERIOD_BEGIN', 'PERIOD_END','STATE']

# Count the number of duplicate rows based on our key
num_duplicates = df.duplicated(subset=key_columns).sum()

if num_duplicates == 0:
    print("✅ Success: No duplicate rows found.")
else:
    print(f"⚠️ Found {num_duplicates} duplicate rows. Removing them now.")
    df.drop_duplicates(subset=key_columns, keep='first', inplace=True)

✅ Success: No duplicate rows found.


In [11]:
df.shape

(3097102, 58)

## 5. Check for Missingness (Missing Values)
Finally, let's get a report of missing values in our dataset. This will help us decide how to handle them in the next stage of our analysis.

In [12]:
# Calculate the number of missing values in each column
missing_values = df.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (missing_values / len(df)) * 100

# Create a report
missing_report = pd.DataFrame({
    'num_missing': missing_values,
    'pct_missing': missing_percentage
})

# Show columns with at least one missing value, sorted by percentage
print("--- Missing Values Report ---")
display(missing_report[missing_report['num_missing'] > 0].sort_values(by='pct_missing', ascending=False))

--- Missing Values Report ---


,num_missing,pct_missing
CITY,3097102,100.000000
MONTHS_OF_SUPPLY,3097102,100.000000
MONTHS_OF_SUPPLY_MOM,3097102,100.000000
MONTHS_OF_SUPPLY_YOY,3097102,100.000000
PRICE_DROPS_YOY,3097102,100.000000
PRICE_DROPS,3097102,100.000000
PRICE_DROPS_MOM,3097102,100.000000
INVENTORY_YOY,676963,21.857950
MEDIAN_LIST_PPSF_YOY,617045,19.923302
NEW_LISTINGS_YOY,615224,19.864506


In [13]:
# --- Create a copy to work on ---
df_cleaned = df.copy()
print(f"Original DataFrame shape: {df_cleaned.shape}")

# Ensure data is sorted for time-series operations
df_cleaned.sort_values(by=['ZIP_CODE', 'PERIOD_BEGIN'], inplace=True)

Original DataFrame shape: (3097102, 58)


### 5.1. Drop 100% Missing Columns
It does not give any value to our case study

In [14]:
# === Step 1: Drop Unusable Columns (100% Missing) ===
columns_to_drop = [
    'CITY', 'MONTHS_OF_SUPPLY', 'MONTHS_OF_SUPPLY_MOM', 'MONTHS_OF_SUPPLY_YOY',
    'PRICE_DROPS', 'PRICE_DROPS_MOM', 'PRICE_DROPS_YOY'
]
df_cleaned.drop(columns=columns_to_drop, inplace=True, errors='ignore')
print(f"\nStep 1: Dropped {len(columns_to_drop)} unusable columns.")



Step 1: Dropped 7 unusable columns.


### 5.2. Reconstruct Derived Metrics (YOY)
For all Year-over-Year (YOY) and Month-over-Month (MOM) columns, we will first attempt to recalculate them from their raw source columns. This is the most accurate way to fill them in.



In [15]:
# === Step 2: Reconstruct All Derived Metrics (YOY & MOM) ===
# Dictionary of {derived_metric: raw_metric}
reconstruction_map = {
    # Year-over-Year (shift 4 quarters)
    'MEDIAN_SALE_PRICE_YOY': 'MEDIAN_SALE_PRICE',
    'HOMES_SOLD_YOY': 'HOMES_SOLD',
    'PENDING_SALES_YOY': 'PENDING_SALES',
    'NEW_LISTINGS_YOY': 'NEW_LISTINGS',
    'INVENTORY_YOY': 'INVENTORY',
    'MEDIAN_DOM_YOY': 'MEDIAN_DOM',
    'MEDIAN_LIST_PRICE_YOY': 'MEDIAN_LIST_PRICE',
    'MEDIAN_PPSF_YOY': 'MEDIAN_PPSF',
    'MEDIAN_LIST_PPSF_YOY': 'MEDIAN_LIST_PPSF',
    'AVG_SALE_TO_LIST_YOY': 'AVG_SALE_TO_LIST',
    'SOLD_ABOVE_LIST_YOY': 'SOLD_ABOVE_LIST',
}

for derived_col, raw_col in reconstruction_map.items():
    if raw_col in df_cleaned.columns and derived_col in df_cleaned.columns:
        shift_period = 4 if '_YOY' in derived_col else 1
        
        # Get the value from the previous period
        last_period_val = df_cleaned.groupby('ZIP_CODE')[raw_col].shift(shift_period)
        
        # Recalculate the percentage change
        recomputed_val = ((df_cleaned[raw_col] - last_period_val) / last_period_val) * 100
        
        # Fill missing values in the derived column with our calculation
        df_cleaned[derived_col] = df_cleaned[derived_col].fillna(recomputed_val)

print("Step 2: Reconstructed all YOY metrics.")



Step 2: Reconstructed all YOY metrics.


### 5.3 Drop MoM data:
A _MOM column would imply a comparison to the previous month, but that data doesn't exist. Instead, the comparison is to the previous quarter. This makes the _MOM label misleading.

In [16]:
# Find all columns that contain '_MOM' in their name
mom_columns = [col for col in df_cleaned.columns if '_MOM' in col]

# Drop these columns from the dataframe
df_cleaned.drop(columns=mom_columns, inplace=True)

print(f"Successfully dropped {len(mom_columns)} misleading '_MOM' columns.")
print(df_cleaned.columns)

Successfully dropped 12 misleading '_MOM' columns.
Index(['PERIOD_BEGIN', 'PERIOD_END', 'PERIOD_DURATION', 'REGION_TYPE',
       'REGION_TYPE_ID', 'TABLE_ID', 'IS_SEASONALLY_ADJUSTED', 'STATE',
       'STATE_CODE', 'PROPERTY_TYPE', 'PROPERTY_TYPE_ID', 'MEDIAN_SALE_PRICE',
       'MEDIAN_SALE_PRICE_YOY', 'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_YOY',
       'MEDIAN_PPSF', 'MEDIAN_PPSF_YOY', 'MEDIAN_LIST_PPSF',
       'MEDIAN_LIST_PPSF_YOY', 'HOMES_SOLD', 'HOMES_SOLD_YOY', 'PENDING_SALES',
       'PENDING_SALES_YOY', 'NEW_LISTINGS', 'NEW_LISTINGS_YOY', 'INVENTORY',
       'INVENTORY_YOY', 'MEDIAN_DOM', 'MEDIAN_DOM_YOY', 'AVG_SALE_TO_LIST',
       'AVG_SALE_TO_LIST_YOY', 'SOLD_ABOVE_LIST', 'SOLD_ABOVE_LIST_YOY',
       'OFF_MARKET_IN_TWO_WEEKS', 'OFF_MARKET_IN_TWO_WEEKS_YOY',
       'PARENT_METRO_REGION', 'PARENT_METRO_REGION_METRO_CODE', 'LAST_UPDATED',
       'ZIP_CODE'],
      dtype='object')


### 5.4. Impute Secondary Metrics: 
For important raw metrics with moderate (5-15%) missingness like INVENTORY or NEW_LISTINGS, we'll use a time-series forward-fill (ffill). This assumes that if data for a period is missing, market conditions were similar to the last known period for that same zip code—a very reasonable approach.

In [17]:
# === Step 4: Impute Secondary Raw Metrics (5-15% Missing) ===
secondary_cols_to_impute = [
    'PENDING_SALES', 'MEDIAN_LIST_PRICE', 'NEW_LISTINGS',
    'MEDIAN_LIST_PPSF', 'INVENTORY'
]
# Use forward-fill within each zip code group
df_cleaned[secondary_cols_to_impute] = df_cleaned.groupby('ZIP_CODE')[secondary_cols_to_impute].ffill()
print("Step 4: Imputed secondary metrics using forward-fill.")

Step 4: Imputed secondary metrics using forward-fill.


### 5.5. Final Cleanup:
For the most essential raw metrics with very low (<5%) missingness (MEDIAN_SALE_PRICE, MEDIAN_DOM), we will drop the few remaining rows. This ensures our core analysis is based purely on observed data.

In [18]:
# === Step 5: Final Cleanup of Essential Metrics (<5% Missing) ===
# These are the absolute minimum columns needed for a valid record
essential_columns = [
    'MEDIAN_SALE_PRICE', 'HOMES_SOLD', 'MEDIAN_DOM'
]
df_cleaned.dropna(subset=essential_columns, inplace=True)
print("Step 5: Dropped remaining rows with missing essential data.")


# === Final Report ===
print(f"\nCleaned DataFrame shape: {df_cleaned.shape}")
print("\n--- Remaining Missing Values Report ---")
missing_report = df_cleaned.isnull().sum()
display(missing_report[missing_report > 0].sort_values(ascending=False))

Step 5: Dropped remaining rows with missing essential data.

Cleaned DataFrame shape: (3062620, 39)

--- Remaining Missing Values Report ---


OFF_MARKET_IN_TWO_WEEKS_YOY    482119
INVENTORY_YOY                  400940
MEDIAN_LIST_PPSF_YOY           344125
NEW_LISTINGS_YOY               333122
MEDIAN_LIST_PRICE_YOY          323604
SOLD_ABOVE_LIST_YOY            227173
PENDING_SALES_YOY              222673
AVG_SALE_TO_LIST_YOY           174714
AVG_SALE_TO_LIST               124422
OFF_MARKET_IN_TWO_WEEKS         90536
MEDIAN_PPSF_YOY                 78662
PENDING_SALES                   78367
INVENTORY                       72565
MEDIAN_DOM_YOY                  50395
MEDIAN_SALE_PRICE_YOY           43345
HOMES_SOLD_YOY                  43314
MEDIAN_PPSF                     30511
SOLD_ABOVE_LIST                 28133
NEW_LISTINGS                    25338
MEDIAN_LIST_PPSF                17447
MEDIAN_LIST_PRICE               14083
dtype: int64

### 5.6. Saved the new dataset

In [19]:
import os

# Define the path for the cleaned data
cleaned_data_path = 'data/cleaned_data/redfin_cleaned.csv'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(cleaned_data_path), exist_ok=True)

# Save the cleaned dataframe to a CSV file
df_cleaned.to_csv(cleaned_data_path, index=False)

print(f"Cleaned dataset successfully saved to: {cleaned_data_path}")

Cleaned dataset successfully saved to: data/cleaned_data/redfin_cleaned.csv
